<a href="https://colab.research.google.com/github/phelipe-sb/Rain-Prediction/blob/main/ML%20Model%20Pipeline/General_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Import libraries
import requests
import time
from datetime import datetime

# Read Files
f = open("project_info.txt", "r")
info_list = eval(f.read())

for x in info_list[1]:
    table_id = x
    
for x in info_list[2]:
    path = x
    
for x in info_list[3]:
    url = x
    
headers = info_list[0]

# Consult Data from API
cities = list(("Sao Paulo", "Carapicuiba", "New York", "Paris", "London", "Rome", "Moscow", "Ottawa", "Hong Kong", "Beijing"))
listdict = list(())
for i in cities:
    querystring = {"q":"{}".format(i)}
    response = requests.request("GET", url, headers=headers, params=querystring)
    var = response.json()
    dic = var['current']
    dic['condition'] = dic['condition']['text']
    dic['city'] = "{}".format(i)
    listdict.append(dic)
    time.sleep(1)

# GCP Credentials
from google.cloud import bigquery
from google.oauth2 import service_account

project_id = 'weather-project-305419'
client = bigquery.Client(project=project_id)
print(f'list:{listdict}')

# Insert data into BigQuery Table Historical
input = []
for x in listdict:
    for i in x.values():
        input.append(i)
        if len(input) < 24:
            pass
        else:
            rows_to_insert = [
            {
            u"last_updated_epoch": u"{}".format(input[0]), 
            u"last_updated": u"{}".format(input[1]),
            u"temp_c": u"{}".format(input[2]),
            u"temp_f": u"{}".format(input[3]),
            u"is_day": u"{}".format(input[4]),
            u"condition": u"{}".format(input[5]),
            u"wind_mph": u"{}".format(input[6]),
            u"wind_kph": u"{}".format(input[7]),
            u"wind_degree": u"{}".format(input[8]),
            u"wind_dir": u"{}".format(input[9]),
            u"pressure_mb": u"{}".format(input[10]),
            u"pressure_in": u"{}".format(input[11]),
            u"precip_mm": u"{}".format(input[12]),
            u"precip_in": u"{}".format(input[13]),
            u"humidity": u"{}".format(input[14]),
            u"cloud": u"{}".format(input[15]),
            u"feelslike_c": u"{}".format(input[16]),
            u"feelslike_f": u"{}".format(input[17]),
            u"vis_km": u"{}".format(input[18]),
            u"vis_miles": u"{}".format(input[19]),
            u"uv": u"{}".format(input[20]),
            u"gust_mph": u"{}".format(input[21]),
            u"gust_kph": u"{}".format(input[22]),
            u"city": u"{}".format(input[23]),
            u"extraction_date": u"{}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
            },
            ]
            errors = client.insert_rows_json(table_id, rows_to_insert)
            input = []

# Insert data into Temporary BigQuery Table
for x in info_list[4]:
    table_id = x

input = []
for x in listdict:
    for i in x.values():
        input.append(i)
        if len(input) < 24:
            pass
        else:
            rows_to_insert = [
            {
            u"last_updated_epoch": u"{}".format(input[0]), 
            u"last_updated": u"{}".format(input[1]),
            u"temp_c": u"{}".format(input[2]),
            u"temp_f": u"{}".format(input[3]),
            u"is_day": u"{}".format(input[4]),
            u"condition": u"{}".format(input[5]),
            u"wind_mph": u"{}".format(input[6]),
            u"wind_kph": u"{}".format(input[7]),
            u"wind_degree": u"{}".format(input[8]),
            u"wind_dir": u"{}".format(input[9]),
            u"pressure_mb": u"{}".format(input[10]),
            u"pressure_in": u"{}".format(input[11]),
            u"precip_mm": u"{}".format(input[12]),
            u"precip_in": u"{}".format(input[13]),
            u"humidity": u"{}".format(input[14]),
            u"cloud": u"{}".format(input[15]),
            u"feelslike_c": u"{}".format(input[16]),
            u"feelslike_f": u"{}".format(input[17]),
            u"vis_km": u"{}".format(input[18]),
            u"vis_miles": u"{}".format(input[19]),
            u"uv": u"{}".format(input[20]),
            u"gust_mph": u"{}".format(input[21]),
            u"gust_kph": u"{}".format(input[22]),
            u"city": u"{}".format(input[23]),
            u"extraction_date": u"{}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
            },
            ]
            print(f'rows_to_insert:{rows_to_insert}')
            errors = client.insert_rows_json(table_id, rows_to_insert)
            input = []

query_string = """
DELETE
FROM (SELECT *
            , RANK() OVER(PARTITION BY last_updated, city ORDER BY last_updated, city) AS rank
            FROM `weather-project-305419.Daily_Weather.Weather-Info-Temp`) as rank_weather
WHERE rank > 3
"""

log = client.query(query_string).result()

time.sleep(480)

# Execute Query and Create DataFrame
query_string = """
SELECT *
FROM (SELECT *
            , ROW_NUMBER() OVER(PARTITION BY last_updated, city ORDER BY last_updated, city) AS row
            FROM `weather-project-305419.Daily_Weather.Weather-Info-Temp`) as row_weather
WHERE row = 1
"""

df = (
    client.query(query_string)
    .result()
    .to_dataframe()
)


df = pd.DataFrame(listdict)
cidades = ['Paris', 'Sao Paulo', 'Carapicuiba', 'New York', 'Otawwa', 'London', 'Rome', 'Moscow'
        , 'Hong Kong', 'Beijing']
df = df.loc[df['city'].isin(cidades)]


# This conditions means that it's raining in the moment
raining_list = ['Light rain', 'Light rain shower', 'Light drizzle', 'Heavy rain', 'Moderate rain', 'Patchy light rain with thunder'
            , 'Moderate or heavy rain shower', 'Patchy light rain', 'Torrential rain shower', 'Moderate rain at times'
            , 'Moderate or heavy rain with thunder']

# Creating the target variable
df['target'] = df['condition'].apply(lambda x: 1 if x in raining_list else 0)
df['target'].unique()

# Get Dummies
x = df[['city', 'temp_c', 'is_day', 'condition', 'wind_kph', 'wind_degree'
          , 'pressure_mb', 'precip_mm', 'humidity', 'cloud', 'feelslike_c', 'vis_km', 'uv'
          , 'gust_kph', 'target']].copy()

x = pd.get_dummies(x, columns=['condition', 'city'])
x.head()

# Open Mask File for algorithm layout
df_mask = pickle.load(open('df_mask', 'rb'))
x = df_mask.append(x)
x = x.fillna(0)

# Open Algorithm Object and apply on DataFrame
model = pickle.load(open('model_v1', 'rb'))
pred = model.predict(x)
results = model.predict_proba(x)

# Transform in probability
prob_list = []
for p in results:
  prob_list.append(1 - p[0])

# Create Prediction DataFrame
df['prob_chuva'] = pd.DataFrame(prob_list)
df['pred'] = pd.DataFrame(pred)
df['last_updated'] = df['last_updated'].astype('datetime64[ns]')
df['prediction_time'] = df['last_updated'] + pd.Timedelta(hours=4)
df = df[['prediction_time', 'city', 'prob_chuva', 'pred']]

for x in info_list[5]:
    table_id = x

input = []
listdict = df.values.to_list()
for x in listdict:
    for i in x.values():
        input.append(i)
        if len(input) < 24:
            pass
        else:
            rows_to_insert = [
            {
            u"last_updated_epoch": u"{}".format(input[0]), 
            u"last_updated": u"{}".format(input[1]),
            u"temp_c": u"{}".format(input[2]),
            u"temp_f": u"{}".format(input[3]),
            u"is_day": u"{}".format(input[4]),
            u"condition": u"{}".format(input[5]),
            u"wind_mph": u"{}".format(input[6]),
            u"wind_kph": u"{}".format(input[7]),
            u"wind_degree": u"{}".format(input[8]),
            u"wind_dir": u"{}".format(input[9]),
            u"pressure_mb": u"{}".format(input[10]),
            u"pressure_in": u"{}".format(input[11]),
            u"precip_mm": u"{}".format(input[12]),
            u"precip_in": u"{}".format(input[13]),
            u"humidity": u"{}".format(input[14]),
            u"cloud": u"{}".format(input[15]),
            u"feelslike_c": u"{}".format(input[16]),
            u"feelslike_f": u"{}".format(input[17]),
            u"vis_km": u"{}".format(input[18]),
            u"vis_miles": u"{}".format(input[19]),
            u"uv": u"{}".format(input[20]),
            u"gust_mph": u"{}".format(input[21]),
            u"gust_kph": u"{}".format(input[22]),
            u"city": u"{}".format(input[23]),
            u"extraction_date": u"{}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
            },
            ]
            print(f'rows_to_insert:{rows_to_insert}')
            errors = client.insert_rows_json(table_id, rows_to_insert)
            input = []

DefaultCredentialsError: ignored

NameError: ignored